# TMDB Semantic Search

In [ ]:
from IPython.display import display
import os
import clip
import torch
from aperturedb.CommonLibrary import (
    create_connector
)
from aperturedb.Utils import Utils


descriptor_set = "ViT-B/16"

# Choose the model to be used.
device = "cuda" if torch.cuda.is_available() else "cpu"
clip.available_models()

# We change the descriptor set here since we are looking for images in the CLIP descriptor set
model, preprocess = clip.load(descriptor_set, device=device)

In [ ]:
import os
os.environ["APERTUREDB_KEY"] = "WzEsMSwicmFnLXRlc3Qtcmg4ZWNlZmwuZmFybTAwMDQuY2xvdWQuYXBlcnR1cmVkYXRhLmRldiIsInBuZmVKdnR5cXVwSDdsZ1k4RE5pOVEzZWhUV3kxYTAybXB0Il0="

In [ ]:
from aperturedb.Constraints import Constraints
from aperturedb.Connector import Connector
import json
from aperturedb.Images import Images

def find_movie_posters(db: Connector,
    display_input:bool = True,
    constraints: Constraints = None,
    search_set_name: str = None,
    embedding: bytes = None,
    k_neighbors: int = 10,
    output_limit:int = 10,
    log_raw_output:bool = False) -> Images:
    """
    Find similar images to the input embedding.
    """

    #First implementation for images search.
    find_image_command = {
        # Retrieve the images associated with the results above.
        "FindImage": {
            # Find images connected to the descriptors returned above.
            "is_connected_to": {
                "ref": 1,
            },
            "group_by_source": True, # Group the results by the source descriptor.
            "results": {
                "list": ["_uniqueid"],
                "limit": k_neighbors
            }
        }
    }
    if constraints is not None:
        print(constraints.constraints)
        find_image_command["FindImage"]["constraints"] = constraints.constraints
        find_image_command["FindImage"]["results"]["list"].extend(list(constraints.constraints.keys()))

    # This ApertureDB query finds images that are similary to the query image.
    # Read about the [FindDescriptor](https://docs.aperturedata.io/query_language/Reference/descriptor_commands/desc_commands/FindDescriptor) command.
    q = [{
        # Find descriptors similar to the input descriptor.
        "FindDescriptor": {
            "set": search_set_name,
            "k_neighbors": 100,
            "_ref": 1,
            "distances": True,
            "results": {
                "all_properties": True,
            },
            "constraints": {
                "source": ["==", "image"]
            }
        }
    }, find_image_command ]

    # Run the query.
    # As additional input, include the descriptor data generated from our query image above.
    responses, images = db.query(q, [embedding])
    # assert len(descriptors) == len(images), f"The number of descriptors and images should be the same {responses}"
    print(f"{responses=}")

    ordered_images = []
    # Compose an ordered response of the images
    descriptors = responses[0]['FindDescriptor']['entities']
    images = responses[1]['FindImage']['entities']
    for descriptor in descriptors:
        desc_id = descriptor['_uniqueid']
        if desc_id in images and len(images[desc_id]) > 0:
            ordered_images.append(
                {**images[desc_id][0]}
            )
            if len(ordered_images) >= output_limit:
                break

    if log_raw_output:
        print(f"{json.dumps(responses, indent=2)}")

    imgs = Images(db, response=ordered_images)
    return imgs

def find_movies(db: Connector,
                display_input:bool = True,
                movie_constraints: Constraints = None,
                search_set_name: str = None,
                embedding: bytes = None,
                k_neighbors: int = 10,
                output_limit:int = 10,
                log_raw_output:bool = False) -> Images:


    """
    Find similar images to the input embedding.
    """

    find_descriptor_image_command = {
        # Find descriptors similar to the input descriptor.
        "FindDescriptor": {
            "set": search_set_name,
            "k_neighbors": 10000,
            "_ref": 1,
            "results": {
                "list": ["_uniqueid"],
            },
            "constraints": {
                "source": ["==", "image"]
            }
        }
    }

    #First implementation for images search.
    find_image_command = {
        # Retrieve the images associated with the results above.
        "FindImage": {
            # Find images connected to the descriptors returned above.
            "is_connected_to": {
                "ref": 1,
            },
            "group_by_source": True, # Group the results by the source descriptor.
            "results": {
                "list": ["_uniqueid", "title"],
                "limit": k_neighbors
            }
        }
    }

    find_descriptor_title_command = {
        # Find descriptors similar to the input descriptor.
        "FindDescriptor": {
            "set": search_set_name,
            "k_neighbors": 10000,
            "_ref": 2,
            "results": {
                "list": ["_uniqueid"],
            },
            "constraints": {
                "source": ["==", "tagline"]
            }
        }
    }

    find_descriptor_tagline_command = {
        # Find descriptors similar to the input descriptor.
        "FindDescriptor": {
            "set": search_set_name,
            "k_neighbors": 10000,
            "_ref": 3,
            "results": {
                "list": ["_uniqueid"],
            },
            "constraints": {
                "source": ["==", "tagline"]
            }
        }
    }

    #First implementation for images search.
    find_movie_command_title = {
        # Retrieve the images associated with the results above.
        "FindEntity": {
            "with_class": "Movie",
            # Find images connected to the descriptors returned above.
            "is_connected_to": {
                "ref": 2,
            },
            "group_by_source": True, # Group the results by the source descriptor.
            "results": {
                "list": ["title", "tagline", "popularity", "vote_average"],
                "limit": k_neighbors
            }
        }
    }
    if movie_constraints is not None:
        print(movie_constraints.constraints)
        find_movie_command_title["FindEntity"]["constraints"] = movie_constraints.constraints


    #First implementation for images search.
    find_movie_command_tagline = {
        # Retrieve the images associated with the results above.
        "FindEntity": {
            "with_class": "Movie",
            # Find images connected to the descriptors returned above.
            "is_connected_to": {
                "ref": 3,
            },
            "group_by_source": True, # Group the results by the source descriptor.
            "results": {
                "list": ["title", "tagline", "popularity", "vote_average"],
                "limit": k_neighbors
            }
        }
    }
    if movie_constraints is not None:
        print(movie_constraints.constraints)
        find_movie_command_tagline["FindEntity"]["constraints"] = movie_constraints.constraints

    # This ApertureDB query finds images that are similary to the query image.
    # Read about the [FindDescriptor](https://docs.aperturedata.io/query_language/Reference/descriptor_commands/desc_commands/FindDescriptor) command.
    q = [find_descriptor_image_command, find_image_command,
         find_descriptor_title_command, find_movie_command_title,
         find_descriptor_tagline_command, find_movie_command_tagline]


    #print(q)

    # Run the query.
    # As additional input, include the descriptor data generated from our query image above.
    responses, images = db.query(q, [embedding, embedding, embedding])
    #db.print_last_response()

    ordered_images = []
    # Compose an ordered response of the images
    descriptors = responses[0]['FindDescriptor']['entities']
    images = responses[1]['FindImage']['entities']
    for descriptor in descriptors:
        desc_id = descriptor['_uniqueid']
        if desc_id in images and len(images[desc_id]) > 0:
            ordered_images.append(
                {**images[desc_id][0]}
            )
            if len(ordered_images) >= output_limit:
                break

    print("##### Matches based on titles ####\n")
    descriptors = responses[2]['FindDescriptor']['entities']
    movies = responses[3]['FindEntity']['entities']
    prop_list = []
    for descriptor in descriptors:
        desc_id = descriptor['_uniqueid']
        if desc_id in movies and len(movies[desc_id]) > 0:
            prop_list.append(movies[desc_id][0])
    display(pd.json_normalize(prop_list))

    if log_raw_output:
        print(f"{json.dumps(responses, indent=2)}")

    print("\n##### Matches based on taglines #### ")
    descriptors = responses[4]['FindDescriptor']['entities']
    movies = responses[5]['FindEntity']['entities']
    prop_list = []
    for descriptor in descriptors:
        desc_id = descriptor['_uniqueid']
        if desc_id in movies and len(movies[desc_id]) > 0:
            prop_list.append(movies[desc_id][0])

    display(pd.json_normalize(prop_list))

    if log_raw_output:
        print(f"{json.dumps(responses, indent=2)}")

    print("##### Matches based on images #### ")
    imgs = Images(db, response=ordered_images)
    return imgs


In [ ]:
client=create_connector()
utils = Utils(client)
utils.summary()

## Find movie posters by multimodal search of images

In [ ]:
# Prompt for free language input
inp = input("Enter a search term as described above: ")
# action scenes in superhero movies
# blue aliens

search_tokens = clip.tokenize([f"a photo of {inp}"]).to(device)
search_embeddings = model.encode_text(search_tokens)

if device == "cuda":
    search_embeddings = search_embeddings.float()
    blobs = search_embeddings[0].cpu().detach().numpy().tobytes()
else:
    blobs = search_embeddings[0].detach().numpy().tobytes()

imgs = find_movie_posters(client, embedding=blobs, k_neighbors=1000, output_limit=10, search_set_name="wf_embeddings_clip")

slider, table = imgs.inspect()
display(slider, table)

## Find movies and their posters by search in title, tagline, images (multimodal RAG)

In [ ]:
# Prompt for free language input
inp = input("Enter a search term as described above")
# action scenes in superhero movies
# blue aliens

search_tokens = clip.tokenize([f"a photo of {inp}"]).to(device)
search_embeddings = model.encode_text(search_tokens)

if device == "cuda":
    search_embeddings = search_embeddings.float()
    blobs = search_embeddings[0].cpu().detach().numpy().tobytes()
else:
    blobs = search_embeddings[0].detach().numpy().tobytes()

imgs = find_movies(client, embedding=blobs, k_neighbors=10, output_limit=10, search_set_name="wf_embeddings_clip")

slider, table = imgs.inspect()
display(slider, table)

## Vector search with additional metadata constraints (vector + graph)

In [ ]:
from aperturedb.Constraints import Constraints

# Prompt for free language input
inp = input("Enter a search term as described above: ")
# action scenes in superhero movies
# blue aliens

const = Constraints()

# We not only want to filter using some metadata key,value property here,
# but want to retrieve the corresponding image in one query
const.greater("popularity", 20)

search_tokens = clip.tokenize([f"a photo of {inp}"]).to(device)
search_embeddings = model.encode_text(search_tokens)

if device == "cuda":
    search_embeddings = search_embeddings.float()
    blobs = search_embeddings[0].cpu().detach().numpy().tobytes()
else:
    blobs = search_embeddings[0].detach().numpy().tobytes()

imgs = helper.find_movies(client, movie_constraints=const, embedding=blobs, k_neighbors=10, output_limit=10, search_set_name=descriptor_set)

slider, table = imgs.inspect()
display(slider, table)